# Exploring the Influence of Social Life on Academics


### Silvia Camara, Brenna Leahy, Maria Gund


Our project will be based on a dataset from Kaggle that explores the effect of socioeconomic factors on students grades from a survey of students’ math and portuguese language courses in secondary school. The dataset contains data gathered from Portuguese students who were surveyed on several different aspects of their life, including familial and social relations as well as their academic habits and socio-economic status.We plan to use the data set to predict a student's final grade in a class based on different social factors, including the amount of time he or she spends going out with friends, the quality of their family relationships, whether they take part in after-school activities, as well as the size of their family. Using several features, we want to see if there is a correlation between a student's social life and their grades. We also want to see if there is one specific feature (or a combination of features) that better predicts whether a student's grade will be above average, average, or below average. 

We plan to explore this topic by using principal component analysis to determine which parts of the data are most useful as well as Naive Bayes to present the data comprehensively and analyze it.


In [13]:
# Allow inline plotting of figures.
%matplotlib inline

# Import useful libraries.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.preprocessing import Imputer

from sklearn.ensemble import RandomForestClassifier

# Read the file into a DataFrame.
math_data = pd.read_csv("student-mat.csv")
port_data = pd.read_csv("student-por.csv")
#data = data.fillna(0)
#data = data.dropna()

In [14]:
# We need to fix what numbers go in the arrays to match our number of features and class

X_math = math_data.iloc[:, :2]  # features
Y_math = math_data.iloc[:, -1]  # class

X_port = port_data.iloc[:, :2]  # features
Y_port = port_data.iloc[:, -1]  # class

print(X.head())
print()
print(y.head())

  school sex
0     GP   F
1     GP   F
2     GP   F
3     GP   F
4     GP   F

0     6
1     6
2    10
3    15
4    10
Name: G3, dtype: int64


## Math Data Set

In [15]:
# Output the first few rows of the math dataset.
math_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


## Portuguese Data Set

In [16]:
# Output the first few rows of the portuguese dataset.
portug_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


## Maria, you said something about binning the data?

In [17]:
# Reid said Random Forest Classifier is very similar to Decision Trees. We just had to change the classifier, 
# which I already did. Try to make the rest of the code from the Classifiers notebook work with our stuff.
# Focus on getting the math data set running for the sake of simplicty and once we do that we can copy
# and paste it for the other one. 

math_rfc = RandomForestClassifier(max_depth=4)
math_rfc.fit(X_math, Y_math)


# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
x_min, x_max = X_math.iloc[:, 0].min() - 1, X_math.iloc[:, 0].max() + 1
y_min, y_max = X_math.iloc[:, 1].min() - 1, X_math.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = clf_dtc.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(X_math.iloc[:, 0], X_math.iloc[:, 1], c=Y, cmap=cmap_bold)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("Random Forest Classifier")

ValueError: could not convert string to float: 'M'